# Тренирање RL за балансирање Cartpole-а

Овај нотебук је део [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Инспирисан је [званичним PyTorch туторијалом](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) и [овом имплементацијом Cartpole-а у PyTorch-у](https://github.com/yc930401/Actor-Critic-pytorch).

У овом примеру, користићемо RL да обучимо модел за балансирање шипке на колицима која могу да се крећу лево и десно на хоризонталној скали. Користићемо окружење [OpenAI Gym](https://www.gymlibrary.ml/) за симулацију шипке.

> **Напомена**: Код из овог часа можете покренути локално (нпр. из Visual Studio Code-а), у ком случају ће се симулација отворити у новом прозору. Приликом покретања кода онлајн, можда ће бити потребно да направите неке измене у коду, као што је описано [овде](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Почећемо тако што ћемо се уверити да је Gym инсталиран:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Сада ћемо креирати окружење CartPole и видети како да радимо са њим. Окружење има следеће особине:

* **Простор акција** је скуп могућих акција које можемо извршити у сваком кораку симулације  
* **Простор опажања** је простор опажања која можемо направити  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Хајде да видимо како симулација функционише. Следећа петља покреће симулацију све док `env.step` не врати заставицу за завршетак `done`. Насумично ћемо бирати акције користећи `env.action_space.sample()`, што значи да ће експеримент вероватно врло брзо пропасти (CartPole окружење се завршава када брзина CartPole-а, његова позиција или угао пређу одређене границе).

> Симулација ће се отворити у новом прозору. Можете покренути код више пута и видети како се понаша.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Можете приметити да запажања садрже 4 броја. То су:
- Позиција колица
- Брзина колица
- Угао шипке
- Стопа ротације шипке

`rew` је награда коју добијамо на сваком кораку. У окружењу CartPole добијате 1 поен за сваки корак симулације, а циљ је максимизовати укупну награду, односно време током којег CartPole може да одржи равнотежу без пада.

Током учења путем појачања, наш циљ је да обучимо **политику** $\pi$, која ће нам за свако стање $s$ рећи коју акцију $a$ треба да предузмемо, што у суштини значи $a = \pi(s)$.

Ако желите вероватносно решење, можете замислити политику као скуп вероватноћа за сваку акцију, односно $\pi(a|s)$ би значило вероватноћу да треба да предузмемо акцију $a$ у стању $s$.

## Метод градијента политике

У најједноставнијем RL алгоритму, који се зове **Градијент политике**, обучићемо неуронску мрежу да предвиди следећу акцију.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Ми ћемо тренирати мрежу извођењем многих експеримената и ажурирањем наше мреже након сваког извођења. Хајде да дефинишемо функцију која ће изводити експеримент и враћати резултате (тзв. **траг**) - сва стања, акције (и њихове препоручене вероватноће) и награде:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Можете покренути једну епизоду са необученом мрежом и приметити да је укупна награда (позната и као дужина епизоде) веома ниска:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Један од изазовних аспеката алгоритма градијента политике је коришћење **дисконтованих награда**. Идеја је да израчунамо вектор укупних награда на сваком кораку игре, и током тог процеса дисконтоваћемо ране награде користећи неки коефицијент $gamma$. Такође нормализујемо резултујући вектор, јер ћемо га користити као тежину да утичемо на нашу обуку:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Хајде да започнемо са правим тренингом! Извршићемо 300 епизода, а у свакој епизоди ћемо урадити следеће:

1. Покренути експеримент и прикупити траг.
2. Израчунати разлику (`градијенте`) између предузетих акција и предвиђених вероватноћа. Што је разлика мања, то смо сигурнији да смо предузели исправну акцију.
3. Израчунати дисконтоване награде и помножити градијенте са дисконтованим наградама - то ће осигурати да кораци са већим наградама имају већи утицај на коначни резултат од оних са мањим наградама.
4. Очекиване циљне акције за нашу неуронску мрежу делимично ће бити преузете из предвиђених вероватноћа током извршавања, а делимично из израчунатих градијената. Користићемо параметар `alpha` да одредимо у којој мери се градијенти и награде узимају у обзир - ово се назива *стопа учења* алгоритма за појачано учење.
5. На крају, тренирамо нашу мрежу на основама стања и очекиваних акција, и понављамо процес.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Сада хајде да покренемо епизоду са рендеровањем да видимо резултат:


In [ ]:
_ = run_episode(render=True)

Надамо се да сада можете видети да штап прилично добро одржава равнотежу!

## Модел Актор-Критичар

Модел Актор-Критичар представља даљи развој метода градијената политике, у коме градимо неуронску мрежу која учи и политику и процењене награде. Мрежа ће имати два излаза (или се може посматрати као две одвојене мреже):
* **Актор** ће препоручивати коју акцију треба предузети тако што ће нам дати расподелу вероватноће стања, као у моделу градијента политике.
* **Критичар** ће процењивати каква би награда могла бити од тих акција. Он враћа укупно процењене награде у будућности за дато стање.

Хајде да дефинишемо такав модел:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Морали бисмо мало изменити наше функције `discounted_rewards` и `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Сада ћемо покренути главну петљу за тренирање. Користићемо процес ручног тренирања мреже рачунањем одговарајућих функција губитка и ажурирањем параметара мреже:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Закључак

У овом демо примеру видели смо два алгоритма за учење путем појачања: једноставан градијент политике и сложенији актер-критичар. Можете приметити да ти алгоритми раде са апстрактним појмовима као што су стање, акција и награда - што значи да се могу применити у веома различитим окружењима.

Учење путем појачања нам омогућава да научимо најбољу стратегију за решавање проблема само на основу посматрања коначне награде. Чињеница да нам нису потребни означени скупови података омогућава нам да више пута понављамо симулације како бисмо оптимизовали наше моделе. Ипак, још увек постоји много изазова у области учења путем појачања, које можете истражити ако одлучите да се више посветите овом занимљивом делу вештачке интелигенције.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
